# Analysis and Visualisation



For accurate analysis, we use the original dataset for the actual values of money, which existed before we altered their values and distributions in the previous explroatory data analysis.

In [ ]:
import sys
sys.path.insert(0, ".")

from pathlib import Path
import db_utils
import pandas as pd

db_utils

df = pd.read_csv('loan_payments.csv')
df

## Current State of the Loans




We will query the data to check the current state of the payments:


What percentage of the loans are recovered against the investor funding and the total amount funded?

we can visualise the results on an appropriate graph.



In [ ]:


for each row in df:
    percentage_of_loan_recovered_so_far = 100*total_payment/loan_amount
    total_funded = funded_amount + funded_amount_inv
    #  create scatter graph with x-axis of total_funded and y-axis of percentage of loan recovered so far

We can also visualise what percentage of the total amount would be recovered up to 6 months' in the future:

In [ ]:
percentage_of_the_total_amount_recovered_up_to_6_months_in_the_future = 100*(total_payment+instalment*6)/loan_amount
